In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad
from torch.autograd import Variable

import os
import time
import utils
import numpy as np

from tqdm import tqdm, trange
import matplotlib.pyplot as plt

import networks
import importlib

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
seed = 1234
torch.set_default_dtype(torch.float32)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
# torch.set_default_tensor_type(torch.DoubleTensor)
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


# Data Sampling
Here in our case, the system is European Call Option PDE and the physical information about the system consists of Boundary Value conditions, final Value conditions and the PDE itself.

In [3]:
K = 50
r = 0.035
sigma = 0.2
T = 1
S_range = [0, int(5*K)]
t_range = [0, T]
gs = lambda x: np.fmax(x-K, 0)

# Build Neural Network

In [4]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers):
        super(Net, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size, hidden_size)])
        self.layers.extend([nn.Linear(hidden_size, hidden_size) for _ in range(n_layers - 1)])
        self.output = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        for layer in self.layers:
            x = self.relu(layer(x))
        x = self.output(x)
        return x



# layers = [2, 50, 50, 50, 50, 50, 50, 50, 50, 1] # Network structure
net = Net(2, 50,1,3).to(device) #  Network initialization

In [5]:
n_epochs = 30000
lossFunction = nn.MSELoss()

from torchimize.functions import lsq_lma
# coeffs_list = lsq_lma(fnn.parameters(), function=lossFunction)
optimizer = optim.Adam(net.parameters(), lr=0.00003)

x_f_s = torch.tensor(0.).float().to(device).requires_grad_(True)
x_label_s = torch.tensor(0.).float().to(device).requires_grad_(True)
optimizer_adam_weight = torch.optim.Adam([x_f_s] + [x_label_s], lr=0.0003)


samples = {"pde": 5000, "bc":500, "fc":500}

# Modelling


In [6]:
loss_hist = []
start_time = time.time()

for epoch in range(n_epochs):
    bc_st_train, bc_v_train, n_st_train, n_v_train = \
    utils.trainingData(K, 
                       r, 
                       sigma, 
                       T, 
                       S_range[-1], 
                       S_range, 
                       t_range, 
                       gs, 
                       samples['bc'], 
                       samples['fc'], 
                       samples['pde'], 
                       RNG_key=123)
    # save training data points to tensor and send to device
    n_st_train = torch.from_numpy(n_st_train).float().requires_grad_().to(device)
    n_v_train = torch.from_numpy(n_v_train).float().to(device)
    
    bc_st_train = torch.from_numpy(bc_st_train).float().to(device)
    bc_v_train = torch.from_numpy(bc_v_train).float().to(device)


    # PDE Round
    y1_hat = net(n_st_train)
    grads = tgrad.grad(y1_hat, n_st_train, grad_outputs=torch.ones(y1_hat.shape).cuda(), retain_graph=True, create_graph=True, only_inputs=True)[0]
    dVdt, dVdS = grads[:, 0].view(-1, 1), grads[:, 1].view(-1, 1)
    grads2nd = tgrad.grad(dVdS, n_st_train, grad_outputs=torch.ones(dVdS.shape).cuda(), create_graph=True, only_inputs=True)[0]
    d2VdS2 = grads2nd[:, 1].view(-1, 1)
    S1 = n_st_train[:, 1].view(-1, 1)
    pde_loss = lossFunction(-dVdt, 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*y1_hat)
    
    # conditions Round
    y21_hat = net(bc_st_train)
    bc_loss = lossFunction(bc_v_train, y21_hat)
    
    # Backpropagation and Update
    optimizer.zero_grad()
    combined_loss = torch.exp(-x_f_s.detach()) * pde_loss + torch.exp(-x_label_s.detach()) * bc_loss + x_f_s.detach() + x_label_s.detach()
    combined_loss.backward()
    optimizer.step()
    
    # update the weight
    optimizer_adam_weight.zero_grad()
    loss = 1/(2*torch.exp(-x_f_s)) * pde_loss.detach() + 1/(2*torch.exp(-x_label_s)) * bc_loss.detach() + x_label_s + x_f_s
    loss.backward()
    optimizer_adam_weight.step()
    
    # record the loss
    mse_loss = pde_loss + bc_loss
    loss_hist.append(mse_loss.item())
    if epoch % 500 == 0:
        print(f'{epoch}/{n_epochs} PDE Loss: {pde_loss.item():.5f}, BC Loss: {bc_loss.item():.5f}, nn_loss: {combined_loss.item():5f}, total loss: {mse_loss.item():.5f}, minimum loss: {min(loss_hist):.5f}')
        print(f'the weight is {torch.exp(-x_f_s.detach()).item():.5f}, {torch.exp(-x_label_s.detach()).item():.5f}')
    pass

end_time = time.time()

print('run time:', end_time - start_time)
print('Adam done!')

0/30000 PDE Loss: 0.00038, BC Loss: 16734.41797, nn_loss: 16734.417969, total loss: 16734.41797, minimum loss: 16734.41797
the weight is 1.00030, 1.00030
500/30000 PDE Loss: 0.00055, BC Loss: 9564.14648, nn_loss: 10890.958984, total loss: 9564.14746, minimum loss: 9564.14746
the weight is 1.16218, 1.13899
1000/30000 PDE Loss: 0.00688, BC Loss: 1142.82397, nn_loss: 1380.223999, total loss: 1142.83081, minimum loss: 1142.83081
the weight is 1.35037, 1.20819
1500/30000 PDE Loss: 0.17600, BC Loss: 159.70251, nn_loss: 193.798584, total loss: 159.87852, minimum loss: 159.87852
the weight is 1.57356, 1.21583
2000/30000 PDE Loss: 0.45613, BC Loss: 151.86432, nn_loss: 185.377991, total loss: 152.32045, minimum loss: 152.32045
the weight is 1.84281, 1.22049
2500/30000 PDE Loss: 0.96433, BC Loss: 144.62543, nn_loss: 178.486710, total loss: 145.58975, minimum loss: 145.58975
the weight is 2.16894, 1.22645
3000/30000 PDE Loss: 1.77401, BC Loss: 136.07768, nn_loss: 171.296997, total loss: 137.85168,